In [1]:
import os
import glob

from IPython.display import display, Markdown
import pandas as pd

# Group types of models (experiment type and model type) and pick best performing in terms of f1-score per unit of analysis and report them in a table


In [2]:
results_filepaths = glob.glob('./logged_performance_per_model/*/*agg*.csv')

In [3]:
results_filepaths

['./logged_performance_per_model/AshtonIsNotHere-xlm-roberta-long-base-4096/multilingual_fit_trunc_512_tokens_agg_AshtonIsNotHere_xlm-roberta-long-base-4096-title_and_first_paragraph_metrics.csv',
 './logged_performance_per_model/AshtonIsNotHere-xlm-roberta-long-base-4096/multilingual_fit_trunc_512_tokens_agg_AshtonIsNotHere_xlm-roberta-long-base-4096-title_and_5_sentences_metrics.csv',
 './logged_performance_per_model/AshtonIsNotHere-xlm-roberta-long-base-4096/multilingual_fit_trunc_512_tokens_agg_AshtonIsNotHere_xlm-roberta-long-base-4096-title_metrics.csv',
 './logged_performance_per_model/AshtonIsNotHere-xlm-roberta-long-base-4096/multilingual_fit_trunc_512_tokens_agg_AshtonIsNotHere_xlm-roberta-long-base-4096-raw_text_metrics.csv',
 './logged_performance_per_model/AshtonIsNotHere-xlm-roberta-long-base-4096/multilingual_fit_trunc_512_tokens_agg_AshtonIsNotHere_xlm-roberta-long-base-4096-title_and_first_sentence_each_paragraph_metrics.csv',
 './logged_performance_per_model/AshtonIsN

In [4]:
dfs_list = []
for results_filepath in results_filepaths:
    model_name = results_filepath.split('/')[-2]
    results_df_i = pd.read_csv(results_filepath)
    results_df_i['model_name'] = model_name
    dfs_list.append(results_df_i)

results_df = pd.concat(dfs_list).set_index(['language', 'model_name', 'unit_of_analysis']).sort_index()
results_df.rename(columns={'f1-mico_mean': 'f1-micro_mean', 'f1-mico_std': 'f1-micro_std'}, inplace=True)
results_df

f1-micro_mean  \
language model_name                                 unit_of_analysis                                         
en       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.700232   
                                                    title                                         0.640544   
                                                    title_and_10_sentences                        0.698109   
                                                    title_and_5_sentences                         0.685517   
                                                    title_and_first_paragraph                     0.683085   
                                                    title_and_first_sentence_each_paragraph       0.690757   
fr       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.567744   
                                                    title                                         0.485518   
                                                    title_and_10_sentences                        0.546452   
                                                    title_and_5_sentences                         0.531221   
                                                    title_and_first_paragraph                     0.539356   
                                                    title_and_first_sentence_each_paragraph       0.562919   
ge       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.658114   
                                                    title                                         0.481710   
                                                    title_and_10_sentences                        0.603854   
                                                    title_and_5_sentences                         0.594598   
                                                    title_and_first_paragraph                     0.577792   
                                                    title_and_first_sentence_each_paragraph       0.607281   
it       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.612195   
                                                    title                                         0.473615   
                                                    title_and_10_sentences                        0.572918   
                                                    title_and_5_sentences                         0.552236   
                                                    title_and_first_paragraph                     0.522629   
                                                    title_and_first_sentence_each_paragraph       0.559655   
po       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.669834   
                                                    title                                         0.513491   
                                                    title_and_10_sentences                        0.635858   
                                                    title_and_5_sentences                         0.610558   
                                                    title_and_first_paragraph                     0.583600   
                                                    title_and_first_sentence_each_paragraph       0.637180   
ru       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.534972   
                                                    title                                         0.395327   
                                                    title_and_10_sentences                        0.474459   
                                                    title_and_5_sentences                         0.501664   
                                                    title_and_first_paragraph                     0.463473   
                       

In [5]:
results_df.to_csv('performance_of_models.csv')

### Generate the tables to report

In [6]:
def display_performance_table(df, metric, index_cols=['model_name'], display_=True):
    report_table = df.reset_index().copy()
    report_table['result'] = report_table[f'{metric}_mean'].map(lambda x: f'{x:.2f}') + \
    ' $\pm$ ' + report_table[f'{metric}_std'].map(lambda x: f'{x:.2f}')
    report_table['col_title'] = report_table.unit_of_analysis.str.split('_').str.join(' ') 
    report_table['col_title'] = pd.Categorical(
        report_table.col_title,
        categories=['title', 'title and first paragraph', 'title and 5 sentences', 'title and 10 sentences',
                    'title and first sentence each paragraph', 'raw text'],
        ordered=True)
    report_table = report_table[index_cols + ['col_title', 'result']]\
        .pivot_table(index=index_cols, columns=['col_title'], values=['result'], aggfunc='first', fill_value=0)\
        .droplevel(0, axis=1)

    report_table.columns.names = [None]

    # Highlight best scoring models according to their average
    mean_perf_arr = report_table.applymap(lambda x: float(str(x).split(' ')[0])).to_numpy()
    highlight_mask = mean_perf_arr == mean_perf_arr.max()
    report_table_arr = report_table.to_numpy()  # Note it passes the array by reference
    report_table_arr[highlight_mask] = '**' + report_table_arr[highlight_mask] + '**'

    if display_:
        display(Markdown(report_table.to_markdown()))
    
    return report_table

In [7]:
display_performance_table(df=results_df.loc['en'], metric='f1-micro', index_cols=['model_name'], display_=True)

| model_name                                 | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-------------------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.64 $\pm$ 0.02 | 0.68 $\pm$ 0.01             | 0.69 $\pm$ 0.01         | **0.70 $\pm$ 0.02**      | 0.69 $\pm$ 0.02                           | **0.70 $\pm$ 0.02** |

,title,title and first paragraph,title and 5 sentences,title and 10 sentences,title and first sentence each paragraph,raw text
model_name,,,,,,
AshtonIsNotHere-xlm-roberta-long-base-4096,0.64 $\pm$ 0.02,0.68 $\pm$ 0.01,0.69 $\pm$ 0.01,**0.70 $\pm$ 0.02**,0.69 $\pm$ 0.02,**0.70 $\pm$ 0.02**


### Generate tables for all languages

In [8]:
metrics_to_report = ['f1-micro', 'recall-micro', 'precision-micro', 'roc-auc', 'accuracy']

In [9]:
language_dict = {'en': 'English', 'it': 'Italian', 'fr': 'French', 'po': 'Polish', 'ru': 'Russian', 'ge': 'German'}

In [10]:
results_df

f1-micro_mean  \
language model_name                                 unit_of_analysis                                         
en       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.700232   
                                                    title                                         0.640544   
                                                    title_and_10_sentences                        0.698109   
                                                    title_and_5_sentences                         0.685517   
                                                    title_and_first_paragraph                     0.683085   
                                                    title_and_first_sentence_each_paragraph       0.690757   
fr       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.567744   
                                                    title                                         0.485518   
                                                    title_and_10_sentences                        0.546452   
                                                    title_and_5_sentences                         0.531221   
                                                    title_and_first_paragraph                     0.539356   
                                                    title_and_first_sentence_each_paragraph       0.562919   
ge       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.658114   
                                                    title                                         0.481710   
                                                    title_and_10_sentences                        0.603854   
                                                    title_and_5_sentences                         0.594598   
                                                    title_and_first_paragraph                     0.577792   
                                                    title_and_first_sentence_each_paragraph       0.607281   
it       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.612195   
                                                    title                                         0.473615   
                                                    title_and_10_sentences                        0.572918   
                                                    title_and_5_sentences                         0.552236   
                                                    title_and_first_paragraph                     0.522629   
                                                    title_and_first_sentence_each_paragraph       0.559655   
po       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.669834   
                                                    title                                         0.513491   
                                                    title_and_10_sentences                        0.635858   
                                                    title_and_5_sentences                         0.610558   
                                                    title_and_first_paragraph                     0.583600   
                                                    title_and_first_sentence_each_paragraph       0.637180   
ru       AshtonIsNotHere-xlm-roberta-long-base-4096 raw_text                                      0.534972   
                                                    title                                         0.395327   
                                                    title_and_10_sentences                        0.474459   
                                                    title_and_5_sentences                         0.501664   
                                                    title_and_first_paragraph                     0.463473   
                       

In [11]:
def display_metrics_and_write_to_file(df, grouping_criterion, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    report_tables_dfs_dict = {metric: [] for metric in metrics_to_report}

    for language, results_df in df.groupby(level=0):
        display(Markdown(f'# {language_dict[language]}'))
        
        for metric in metrics_to_report:
            os.makedirs(os.path.join(output_dir, metric), exist_ok=True)

            output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
            output_dir_latex = os.path.join(output_dir, metric, 'latex')
            output_dir_csv = os.path.join(output_dir, metric, 'csv')

            os.makedirs(output_dir_markdown, exist_ok=True)
            os.makedirs(output_dir_latex, exist_ok=True)
            os.makedirs(output_dir_csv, exist_ok=True)

            display(Markdown(f'## {metric}'))

            report_table = display_performance_table(df=results_df, index_cols=grouping_criterion, metric=metric, display_=True)

            # Export as markdown
            markdown_file = open(os.path.join(output_dir_markdown, f"{language_dict[language]}_{metric}.md"), "w")
            report_table.reset_index().to_markdown(markdown_file, index=False)
            markdown_file.close()

            # Export as latex table
            latex_file = open(os.path.join(output_dir_latex, f"{language_dict[language]}_{metric}.tex"), "w")
            report_table.reset_index().to_latex(latex_file, index=False)
            latex_file.close()

            # Export as csv
            report_table.to_csv(os.path.join(output_dir_csv, f"{language_dict[language]}_{metric}.csv"))

            # Stack all languages into single table
            report_table['language'] = language
            report_table = report_table.reset_index().set_index(['language'] + grouping_criterion)

            report_tables_dfs_dict[metric].append(report_table)

    # Report or store unified table
    display(Markdown(f'# All 6 Languages'))
    for metric in metrics_to_report:
        display(Markdown(f'## {metric}'))
        multi_language_report_table_metric = pd.concat(report_tables_dfs_dict[metric])
        display(Markdown(multi_language_report_table_metric.reset_index().to_markdown(index=False)))

        output_dir_markdown = os.path.join(output_dir, metric, 'markdown')
        output_dir_latex = os.path.join(output_dir, metric, 'latex')
        output_dir_csv = os.path.join(output_dir, metric, 'csv')

        # Export as markdown
        markdown_file = open(os.path.join(output_dir_markdown, f"all_6_languages_{metric}.md"), "w")
        multi_language_report_table_metric.reset_index().to_markdown(markdown_file, index=False)
        markdown_file.close()

        # Export as latex table
        latex_file = open(os.path.join(output_dir_latex, f"all_6_languages_{metric}.tex"), "w")
        multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
        latex_file.close()

        # Export as csv
        multi_language_report_table_metric.to_csv(os.path.join(output_dir_csv, f"all_6_languages_{metric}.csv"))

# Per model type

In [12]:
display_metrics_and_write_to_file(df=results_df, grouping_criterion=['model_name'], output_dir='per_model_name_tables')

# English

## f1-micro

| model_name                                 | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-------------------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.64 $\pm$ 0.02 | 0.68 $\pm$ 0.01             | 0.69 $\pm$ 0.01         | **0.70 $\pm$ 0.02**      | 0.69 $\pm$ 0.02                           | **0.70 $\pm$ 0.02** |

/tmp/ipykernel_65223/2331756451.py:30: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  report_table.reset_index().to_latex(latex_file, index=False)


## recall-micro

...

| language   | model_name                                 | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-----------|:-------------------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| en         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.75 $\pm$ 0.01 | 0.77 $\pm$ 0.01             | **0.78 $\pm$ 0.01**     | **0.78 $\pm$ 0.01**      | **0.78 $\pm$ 0.01**                       | **0.78 $\pm$ 0.01** |
| fr         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.67 $\pm$ 0.01 | 0.70 $\pm$ 0.02             | 0.69 $\pm$ 0.01         | 0.70 $\pm$ 0.01          | 0.71 $\pm$ 0.02                           | **0.72 $\pm$ 0.02** |
| ge         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.64 $\pm$ 0.03 | 0.70 $\pm$ 0.01             | 0.71 $\pm$ 0.02         | 0.71 $\pm$ 0.00          | 0.72 $\pm$ 0.01                           | **0.75 $\pm$ 0.00** |
| it         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.65 $\pm$ 0.02 | 0.67 $\pm$ 0.03             | 0.69 $\pm$ 0.01         | 0.70 $\pm$ 0.02          | 0.70 $\pm$ 0.03                           | **0.73 $\pm$ 0.03** |
| po         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.64 $\pm$ 0.01 | 0.69 $\pm$ 0.01             | 0.70 $\pm$ 0.01         | 0.72 $\pm$ 0.01          | 0.73 $\pm$ 0.01                           | **0.75 $\pm$ 0.00** |
| ru         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.63 $\pm$ 0.03 | 0.67 $\pm$ 0.02             | 0.69 $\pm$ 0.03         | 0.67 $\pm$ 0.01          | 0.68 $\pm$ 0.01                           | **0.70 $\pm$ 0.01** |

/tmp/ipykernel_65223/2331756451.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)


## accuracy

| language   | model_name                                 | title           | title and first paragraph   | title and 5 sentences   | title and 10 sentences   | title and first sentence each paragraph   | raw text            |
|:-----------|:-------------------------------------------|:----------------|:----------------------------|:------------------------|:-------------------------|:------------------------------------------|:--------------------|
| en         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.08 $\pm$ 0.01 | 0.07 $\pm$ 0.01             | **0.11 $\pm$ 0.02**     | **0.11 $\pm$ 0.04**      | 0.10 $\pm$ 0.01                           | **0.11 $\pm$ 0.00** |
| fr         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.08 $\pm$ 0.04 | 0.09 $\pm$ 0.03             | 0.08 $\pm$ 0.04         | 0.07 $\pm$ 0.00          | **0.11 $\pm$ 0.01**                       | 0.07 $\pm$ 0.04     |
| ge         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.05 $\pm$ 0.03 | 0.07 $\pm$ 0.04             | 0.06 $\pm$ 0.05         | 0.05 $\pm$ 0.02          | 0.08 $\pm$ 0.04                           | **0.12 $\pm$ 0.01** |
| it         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.06 $\pm$ 0.01 | 0.06 $\pm$ 0.02             | **0.10 $\pm$ 0.03**     | 0.09 $\pm$ 0.02          | 0.08 $\pm$ 0.02                           | **0.10 $\pm$ 0.03** |
| po         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.04 $\pm$ 0.01 | 0.04 $\pm$ 0.02             | 0.06 $\pm$ 0.02         | **0.07 $\pm$ 0.01**      | 0.06 $\pm$ 0.02                           | **0.07 $\pm$ 0.01** |
| ru         | AshtonIsNotHere-xlm-roberta-long-base-4096 | 0.07 $\pm$ 0.03 | 0.12 $\pm$ 0.04             | 0.10 $\pm$ 0.03         | 0.10 $\pm$ 0.05          | 0.12 $\pm$ 0.04                           | **0.17 $\pm$ 0.06** |

/tmp/ipykernel_65223/2331756451.py:60: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  multi_language_report_table_metric.reset_index().to_latex(latex_file, index=False)
